In [1]:
import json
import os
from enum import Enum
from typing import Dict, List, Tuple, Union

import mlx.optimizers as optim
from datasets import load_dataset
from mlx.utils import tree_flatten
from mlx_lm import generate, load
from mlx_lm.tuner import TrainingArgs, datasets, linear_to_lora_layers, train

In [2]:
model_path = "mlx-community/gemma-2-2b-it-4bit"
model, tokenizer = load(model_path)

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
tokenizer.chat_template

"{{ bos_token }}{% if messages[0]['role'] == 'system' %}{{ raise_exception('System role not supported') }}{% endif %}{% for message in messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}{% endif %}{% if (message['role'] == 'assistant') %}{% set role = 'model' %}{% else %}{% set role = message['role'] %}{% endif %}{{ '<start_of_turn>' + role + '\n' + message['content'] | trim + '<end_of_turn>\n' }}{% endfor %}{% if add_generation_prompt %}{{'<start_of_turn>model\n'}}{% endif %}"

In [4]:
tokenizer.chat_template = "{{ bos_token }}{% if messages[0]['role'] == 'system' %}{{ raise_exception('System role not supported') }}{% endif %}{% for message in messages %}{{ '<start_of_turn>' + message['role'] + '\n' + message['content'] | trim + '<end_of_turn><eos>\n' }}{% endfor %}{% if add_generation_prompt %}{{'<start_of_turn>model\n'}}{% endif %}"

In [5]:
dataset_path = "Jofthomas/hermes-function-calling-thinking-V1"

In [ ]:
def preprocess(sample):
    messages = sample["messages"]
    first_message = messages[0]

    # Instead of adding a system message, we merge the content into the first user message
    if first_message["role"] == "system":
        system_message_content = first_message["content"]
        # Merge system content with the first user message
        messages[1]["content"] = (
            system_message_content
            + "Also, before making a call to a function take the time to plan the function to take. Make that thinking process between <think>{your thoughts}</think>\n\n"
            + messages[1]["content"]
        )
        # Remove the system message from the conversation
        messages.pop(0)

    return {"text": tokenizer.apply_chat_template(messages, tokenize=False)}

In [7]:
dataset = load_dataset(dataset_path)
dataset

DatasetDict({
    train: Dataset({
        features: ['conversations'],
        num_rows: 3570
    })
})

In [8]:
dataset = dataset.rename_column("conversations", "messages")
dataset

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 3570
    })
})

In [9]:
dataset = dataset.map(preprocess, remove_columns="messages")
dataset = dataset["train"].train_test_split(0.1)
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 3213
    })
    test: Dataset({
        features: ['text'],
        num_rows: 357
    })
})

In [10]:
print(dataset["train"][8]["text"])

<bos><start_of_turn>human
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.Here are the available tools:<tools> [{'type': 'function', 'function': {'name': 'convert_currency', 'description': 'Convert an amount from one currency to another', 'parameters': {'type': 'object', 'properties': {'amount': {'type': 'number', 'description': 'The amount to be converted'}, 'from_currency': {'type': 'string', 'description': 'The currency to convert from'}, 'to_currency': {'type': 'string', 'description': 'The currency to convert to'}}, 'required': ['amount', 'from_currency', 'to_currency']}}}, {'type': 'function', 'function': {'name': 'play_music', 'description': 'Play music', 'parameters': {'type': 'object', 'properties': {'song': {'type': 'string', 'description': 'The name of the song to play'}, 'artist': {'t

In [11]:
# print(tokenizer.pad_token)
# print(tokenizer.eos_token)

In [12]:
# tokenizer.pad_token

In [13]:
# class ChatmlSpecialTokens(str, Enum):
#     tools = "<tools>"
#     eotools = "</tools>"
#     think = "<think>"
#     eothink = "</think>"
#     tool_call="<tool_call>"
#     eotool_call="</tool_call>"
#     tool_response="<tool_reponse>"
#     eotool_response="</tool_reponse>"
#     pad_token = "<pad>"
#     eos_token = "<eos>"
#     @classmethod
#     def list(cls):
#         return [c.value for c in cls]

In [14]:
# tokenizer.pad_token = ChatmlSpecialTokens.pad_token.value
# tokenizer.pad_token

In [15]:
# tokenizer.additional_special_tokens = ChatmlSpecialTokens.list()
# tokenizer.additional_special_tokens

In [16]:
tokenizer.chat_template = "{{ bos_token }}{% if messages[0]['role'] == 'system' %}{{ raise_exception('System role not supported') }}{% endif %}{% for message in messages %}{{ '<start_of_turn>' + message['role'] + '\n' + message['content'] | trim + '<end_of_turn><eos>\n' }}{% endfor %}{% if add_generation_prompt %}{{'<start_of_turn>model\n'}}{% endif %}"
tokenizer.chat_template

"{{ bos_token }}{% if messages[0]['role'] == 'system' %}{{ raise_exception('System role not supported') }}{% endif %}{% for message in messages %}{{ '<start_of_turn>' + message['role'] + '\n' + message['content'] | trim + '<end_of_turn><eos>\n' }}{% endfor %}{% if add_generation_prompt %}{{'<start_of_turn>model\n'}}{% endif %}"

In [17]:
adapter_path = "adapters_fc"
os.makedirs(adapter_path, exist_ok=True)
adapter_config_path = os.path.join(adapter_path, "adapter_config.json")
adapter_file_path = os.path.join(adapter_path, "adapters.safetensors")

In [18]:
lora_config = {
    "num_layers": 8,
    "lora_parameters": {
        "rank": 16,
        "scale": 64,
        "dropout": 0.05,
    },
}

In [19]:
with open(adapter_config_path, "w") as f:
    json.dump(lora_config, f, indent=4)

In [20]:
training_args = TrainingArgs(
    adapter_file=adapter_file_path,
    iters=1,
    steps_per_eval=50,
)

In [21]:
_ = model.freeze()

In [22]:
linear_to_lora_layers(model, lora_config["num_layers"], lora_config["lora_parameters"])

In [23]:
num_train_params = sum(v.size for _, v in tree_flatten(model.trainable_parameters()))
print(f"Number of trainable parameters: {num_train_params}")

Number of trainable parameters: 983040


In [24]:
_ = model.train()

In [25]:
class Metrics:
    def __init__(self) -> None:
        self.train_losses: List[Tuple[int, float]] = []
        self.val_losses: List[Tuple[int, float]] = []

    def on_train_loss_report(self, info: Dict[str, Union[float, int]]) -> None:
        self.train_losses.append((info["iteration"], info["train_loss"]))

    def on_val_loss_report(self, info: Dict[str, Union[float, int]]) -> None:
        self.val_losses.append((info["iteration"], info["val_loss"]))

In [26]:
metrics = Metrics()

In [27]:
configs = {
    "mask_prompt": False,
    "prompt_feature": "prompt",
    "text_feature": "text",
    "completion_feature": "completion",
    "chat_feature": "messages",
}

In [28]:
dataset["train"]

Dataset({
    features: ['text'],
    num_rows: 3213
})

In [29]:
train_set = datasets.create_dataset(
    dataset["train"],
    tokenizer,
    configs
)

In [30]:
val_set = datasets.create_dataset(
    dataset["test"],
    tokenizer,
    configs
)

In [31]:
train(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    optimizer=optim.Adam(learning_rate=1e-5),
    train_dataset=train_set,
    val_dataset=val_set,
    training_callback=metrics,
)

Starting training..., iters: 1
Iter 1: Val loss 1.781, Val took 143.580s
Iter 1: Train loss 1.746, Learning Rate 1.000e-05, It/sec 0.393, Tokens/sec 145.069, Trained Tokens 3688, Peak mem 20.493 GB
Saved final weights to adapters_fc/adapters.safetensors.


In [32]:
model_lora, _ = load(model_path, adapter_path=adapter_path)

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

In [33]:
model_lora

Model(
  (model): GemmaModel(
    (embed_tokens): QuantizedEmbedding(256000, 2304, group_size=64, bits=4)
    (layers.0): TransformerBlock(
      (self_attn): Attention(
        (q_proj): QuantizedLinear(input_dims=2304, output_dims=2048, bias=False, group_size=64, bits=4)
        (k_proj): QuantizedLinear(input_dims=2304, output_dims=1024, bias=False, group_size=64, bits=4)
        (v_proj): QuantizedLinear(input_dims=2304, output_dims=1024, bias=False, group_size=64, bits=4)
        (o_proj): QuantizedLinear(input_dims=2048, output_dims=2304, bias=False, group_size=64, bits=4)
        (rope): RoPE(256, traditional=False)
      )
      (mlp): MLP(
        (gate_proj): QuantizedLinear(input_dims=2304, output_dims=9216, bias=False, group_size=64, bits=4)
        (down_proj): QuantizedLinear(input_dims=9216, output_dims=2304, bias=False, group_size=64, bits=4)
        (up_proj): QuantizedLinear(input_dims=2304, output_dims=9216, bias=False, group_size=64, bits=4)
      )
      (input_lay

In [34]:
print(dataset["test"][8]["text"])

<bos><start_of_turn>human
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.Here are the available tools:<tools> [{'type': 'function', 'function': {'name': 'calculate_loan_payment', 'description': 'Calculate the monthly payment for a loan', 'parameters': {'type': 'object', 'properties': {'loan_amount': {'type': 'number', 'description': 'The amount of the loan'}, 'interest_rate': {'type': 'number', 'description': 'The interest rate of the loan'}, 'loan_term': {'type': 'integer', 'description': 'The term of the loan in months'}}, 'required': ['loan_amount', 'interest_rate', 'loan_term']}}}, {'type': 'function', 'function': {'name': 'calculate_tip', 'description': 'Calculate the tip amount for a bill', 'parameters': {'type': 'object', 'properties': {'bill_amount': {'type': 'number', 'description': 'Th

In [35]:
prompt="""<bos><start_of_turn>human
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.Here are the available tools:<tools> [{'type': 'function', 'function': {'name': 'convert_currency', 'description': 'Convert from one currency to another', 'parameters': {'type': 'object', 'properties': {'amount': {'type': 'number', 'description': 'The amount to convert'}, 'from_currency': {'type': 'string', 'description': 'The currency to convert from'}, 'to_currency': {'type': 'string', 'description': 'The currency to convert to'}}, 'required': ['amount', 'from_currency', 'to_currency']}}}, {'type': 'function', 'function': {'name': 'calculate_distance', 'description': 'Calculate the distance between two locations', 'parameters': {'type': 'object', 'properties': {'start_location': {'type': 'string', 'description': 'The starting location'}, 'end_location': {'type': 'string', 'description': 'The ending location'}}, 'required': ['start_location', 'end_location']}}}] </tools>Use the following pydantic model json schema for each tool call you will make: {'title': 'FunctionCall', 'type': 'object', 'properties': {'arguments': {'title': 'Arguments', 'type': 'object'}, 'name': {'title': 'Name', 'type': 'string'}}, 'required': ['arguments', 'name']}For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:
<tool_call>
{tool_call}
</tool_call>Also, before making a call to a function take the time to plan the function to take. Make that thinking process between <think>{your thoughts}</think>

Hi, I need to convert 500 USD to Euros. Can you help me with that?<end_of_turn><eos>
<start_of_turn>model
<think>"""

In [36]:
response = generate(model_lora, tokenizer, prompt=prompt, verbose=True, max_tokens=1000)


To convert USD to Euros, I need to use the 'convert_currency' function from the provided tools.  I need to provide the amount to convert, the currency to convert from (USD), and the currency to convert to (Euros).  I should also make sure the amount is a number.
</think>

<tool_call>
{
  'name': 'convert_currency',
  'arguments': {
    'amount': 500,
    'from_currency': 'USD',
    'to_currency': 'EUR'
  }
}
</tool_call> 

Prompt: 460 tokens, 806.327 tokens-per-sec
Generation: 135 tokens, 66.724 tokens-per-sec
Peak memory: 20.493 GB


In [37]:
print(prompt)
print(response)

<bos><start_of_turn>human
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.Here are the available tools:<tools> [{'type': 'function', 'function': {'name': 'convert_currency', 'description': 'Convert from one currency to another', 'parameters': {'type': 'object', 'properties': {'amount': {'type': 'number', 'description': 'The amount to convert'}, 'from_currency': {'type': 'string', 'description': 'The currency to convert from'}, 'to_currency': {'type': 'string', 'description': 'The currency to convert to'}}, 'required': ['amount', 'from_currency', 'to_currency']}}}, {'type': 'function', 'function': {'name': 'calculate_distance', 'description': 'Calculate the distance between two locations', 'parameters': {'type': 'object', 'properties': {'start_location': {'type': 'string', 'description': 'The star